In [1]:
import rectv_gpu
import numpy as np

Make a function for extracting python array pointers that are sent to C++ functions

In [2]:
def getp(a):
    return a.__array_interface__['data'][0]

Initiate basis functions for function decomposition $u=\sum_{j=0}^{m-1}u_j\varphi_j$

In [3]:
def takephi(ntheta):
    m = 8  # number of basis functions
    [x, y] = np.meshgrid(np.arange(-ntheta//2, ntheta//2), np.arange(-m//2, m//2))
    phi = np.zeros([m, 2*ntheta], dtype='float32')
    phi[:, ::2] = np.cos(2*np.pi*x*y/ntheta)/np.sqrt(ntheta)
    phi[:, 1::2] = np.sin(2*np.pi*x*y/ntheta)/np.sqrt(ntheta)
    phi[0] = 0  # symmetric
    return phi

Read numpy array with already filtered data

In [4]:
data = np.load("foambin2.npy")
[ns, ntheta, n] = data.shape
print([ns,ntheta,n])

[16, 2400, 504]


Set the rotation center and projection angles. In this example we have 8 intervals of size $\pi$

In [5]:
rot_center = n//2 
theta = np.linspace(0, 8*np.pi, ntheta, endpoint=False).astype('float32')  

The method is solving the problem $\|\mathcal{R}_\text{apr}u-\text{data}\|_2^2+\lambda_0\Big\|\sqrt{\frac{\partial u}{\partial x}+\frac{\partial u}{\partial y}+\frac{\partial u}{\partial z}+\lambda_1\frac{\partial u}{\partial t}}\Big\|_1\to \min$, $\quad$ where $\mathcal{R}_\text{apr}u=\sum_{j=0}^{m-1}\mathcal{R}u_j\varphi_j$

Init $\lambda_0$ and $\lambda_1$:

In [6]:
lambda0 = 1e-3  # regularization parameter 1
lambda1 = 4  # regularization parameter 2

The minimization problem is solved by the ADMM scheme with using 'niter' outer ADMM iterations and 'titer' inner tomography iterations. 'titer' in practice should be low.    

In [7]:
niter = 16  # number of ADMM iterations
titer = 4  # number of inner tomography iterations

All computations are done on GPUs, where parallelization is done by slices. Variable 'nsp' is the number of slices to process simultaneously by one gpu. 'nsp' is chosen with respect to GPU memory sizes and should be a multiple of 'ns'.     

In [8]:
nsp = 4 # number of slices to process simultaniously by gpu
ngpus = 1 # number of gpus 

Take basis functions for decomosition 

In [9]:
phi = takephi(ntheta) 
m = phi.shape[0] # number of basis functions

Create a class for reconstruction. The class handles CPU and GPU memory allocation in the C++ context.

In [10]:
cl = rectv_gpu.rectv(n, ntheta, m, ns, nsp, ngpus, rot_center, lambda0, lambda1)   

Show relative convergence $u_{n+1}-u_n$ on each iterations

In [11]:
dbg = True

Allocate memory for result and run reconstruction. Note that the function takes pointers (function getp) to memory where python arrays are allocated.

In [12]:
rtv = np.zeros([ns,m,n,n], dtype='float32') # memory for result
cl.run(getp(rtv), getp(data), getp(theta), getp(phi),  niter, titer, dbg)

AttributeError: module '_rectv' has no attribute 'rectv_run'

Save results as tiff

In [ ]:
for k in range(rtv.shape[0]):
    dxchange.write_tiff_stack(rtv[k], 'rec_tv/rec_'+str(k), overwrite=True)